In [1]:
# Cell 1: Configuration and Data Loading
import polars as pl
import numpy as np
from pathlib import Path
from tqdm. auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import catboost as cb
import warnings
import pickle
import zipfile
import json
from datetime import datetime

warnings.filterwarnings('ignore')

# Configuration
CONFIG = {
    'base_path': "/kaggle/input/sih-isro/Data_SIH_2025/",
    'n_sites': 7,
    'n_folds': 5,
    'random_state': 42,
    'targets': ['O3_target', 'NO2_target'],
    'early_stopping': 100,
    'iterations': 2000,
    'depth': 12,
    'learning_rate': 0.03,
    'delhi_lat': 28.6,
    'n_oof_iterations': 2,
}

print("=" * 80)
print("ENHANCED PIPELINE: CTM + SATELLITE CORRECTION + OOF LAGS")
print("=" * 80)
print(f"Configuration:")
print(f"  Iterations: {CONFIG['iterations']}, Depth: {CONFIG['depth']}, LR: {CONFIG['learning_rate']}")
print(f"  OOF Iterations: {CONFIG['n_oof_iterations']}")
print(f"  CV Folds: {CONFIG['n_folds']}")

# Data Loading
print("\n" + "-" * 80)
print("STAGE 0: DATA LOADING AND PREPROCESSING")
print("-" * 80)

def load_data(config):
    """Load and combine all site data."""
    frames = []
    for site_id in tqdm(range(1, config['n_sites'] + 1), desc="Loading sites"):
        path = Path(config['base_path']) / f"site_{site_id}_train_data.csv"
        df = pl.read_csv(path)
        df = df.with_columns(pl.lit(site_id). alias('site_id'))
        frames.append(df)
    
    df = pl.concat(frames)
    
    # DateTime construction
    df = df.with_columns([
        pl.date(pl.col('year'). cast(pl.Int32), 
                pl.col('month').cast(pl.Int32), 
                pl.col('day').cast(pl.Int32)). alias('date')
    ])
    df = df.with_columns([
        (pl.col('date'). cast(pl. Datetime) + 
         pl.duration(hours=pl.col('hour'). cast(pl.Int64))).alias('datetime')
    ])
    
    return df. sort(['site_id', 'datetime'])

def fill_satellite_gaps(df):
    """Fill satellite gaps using daily propagation."""
    sat_cols = ['NO2_satellite', 'HCHO_satellite', 'ratio_satellite']
    
    # Daily aggregation
    daily_sat = (
        df.group_by(['site_id', 'date'])
        .agg([pl.col(col).drop_nulls().first().alias(f'{col}_daily') for col in sat_cols])
    )
    
    df = df.join(daily_sat, on=['site_id', 'date'], how='left')
    
    # Fill strategy
    for col in sat_cols:
        df = df.with_columns([
            pl.coalesce([f'{col}_daily', col]). alias(f'{col}_filled')
        ])
        df = df.with_columns([
            pl.col(f'{col}_filled'). forward_fill().over('site_id')
        ])
        df = df.with_columns([
            pl.col(f'{col}_filled'). backward_fill().over('site_id')
        ])
    
    # Satellite availability flag
    df = df.with_columns([
        pl.col('NO2_satellite'). is_not_null().cast(pl.Int8).alias('sat_available_flag')
    ])
    
    # Drop daily columns
    df = df.drop([f'{col}_daily' for col in sat_cols])
    
    return df

# Load and preprocess
df = load_data(CONFIG)
df = fill_satellite_gaps(df)

print(f"\nData loaded: {df.shape[0]:,} samples x {df.shape[1]} columns")
print(f"Date range: {df['datetime'].min()} to {df['datetime']. max()}")
print(f"Sites: {df['site_id'].n_unique()}")

# Check target availability
for target in CONFIG['targets']:
    n_valid = df[target].is_not_null(). sum()
    pct = n_valid / len(df) * 100
    print(f"{target}: {n_valid:,} valid samples ({pct:.1f}%)")

ENHANCED PIPELINE: CTM + SATELLITE CORRECTION + OOF LAGS
Configuration:
  Iterations: 2000, Depth: 12, LR: 0.03
  OOF Iterations: 2
  CV Folds: 5

--------------------------------------------------------------------------------
STAGE 0: DATA LOADING AND PREPROCESSING
--------------------------------------------------------------------------------


Loading sites:   0%|          | 0/7 [00:00<?, ?it/s]


Data loaded: 171,679 samples x 23 columns
Date range: 2019-07-10 00:00:00 to 2024-06-30 00:00:00
Sites: 7
O3_target: 171,679 valid samples (100.0%)
NO2_target: 171,679 valid samples (100.0%)


In [2]:
# Cell 2: Comprehensive Feature Engineering (100+ Features)
print("\n" + "=" * 80)
print("STAGE 1: COMPREHENSIVE FEATURE ENGINEERING")
print("=" * 80)

def create_all_features(df, config):
    """Create all 100+ features for the model."""
    
    lat_rad = config['delhi_lat'] * np.pi / 180
    
    # =========================================================================
    # CATEGORY 1: TEMPORAL FEATURES (18 features)
    # =========================================================================
    print("\n  [1/10] Creating temporal features...")
    
    # Cyclical encodings
    df = df.with_columns([
        (pl.col('hour') * 2 * np.pi / 24).sin().alias('hour_sin'),
        (pl.col('hour') * 2 * np.pi / 24).cos().alias('hour_cos'),
        pl.col('date').dt.ordinal_day().alias('doy'),
        pl.col('date').dt.weekday().alias('dayofweek'),
    ])
    
    df = df.with_columns([
        (pl.col('doy') * 2 * np.pi / 365).sin().alias('doy_sin'),
        (pl.col('doy') * 2 * np.pi / 365).cos().alias('doy_cos'),
        (pl.col('dayofweek') * 2 * np.pi / 7).sin().alias('dow_sin'),
        (pl.col('dayofweek') * 2 * np.pi / 7).cos().alias('dow_cos'),
    ])
    
    # Weekend and rush hour
    df = df.with_columns([
        (pl.col('dayofweek') >= 5).cast(pl.Int8).alias('is_weekend'),
        ((pl.col('hour') >= 7) & (pl.col('hour') <= 10)).cast(pl.Int8).alias('is_morning_rush'),
        ((pl.col('hour') >= 17) & (pl.col('hour') <= 20)).cast(pl.Int8).alias('is_evening_rush'),
    ])
    
    df = df.with_columns([
        (pl.col('is_morning_rush') | pl.col('is_evening_rush')).cast(pl.Int8).alias('is_rush_hour')
    ])
    
    # Diurnal phase (0-5)
    df = df.with_columns([
        pl.when(pl.col('hour') < 6).then(0)
        .when(pl.col('hour') < 10).then(1)
        .when(pl.col('hour') < 14).then(2)
        .when(pl.col('hour') < 18).then(3)
        .when(pl.col('hour') < 22).then(4)
        .otherwise(5)
        .alias('diurnal_phase')
    ])
    
    # Seasonal indicators
    df = df.with_columns([
        pl.col('month').is_in([12, 1, 2]).cast(pl.Int8).alias('is_winter'),
        pl.col('month').is_in([3, 4, 5]).cast(pl.Int8).alias('is_summer'),
        pl.col('month').is_in([6, 7, 8, 9]).cast(pl.Int8).alias('is_monsoon'),
        pl.col('month').is_in([10, 11]).cast(pl.Int8).alias('is_post_monsoon'),
    ])
    
    # =========================================================================
    # CATEGORY 2: SOLAR GEOMETRY (4 features)
    # =========================================================================
    print("  [2/10] Creating solar geometry features...")
    
    df = df.with_columns([
        (23.45 * np.pi / 180 * (2 * np.pi * (pl.col('doy') - 81) / 365).sin()).alias('declination'),
        ((pl.col('hour') - 12) * 15 * np.pi / 180).alias('hour_angle'),
    ])
    
    df = df.with_columns([
        (np.sin(lat_rad) * pl.col('declination').sin() + 
         np.cos(lat_rad) * pl.col('declination').cos() * pl.col('hour_angle').cos()
        ).clip(0, 1).alias('cos_sza')
    ])
    
    df = df.with_columns([
        pl.col('cos_sza').pow(2).alias('cos_sza_squared'),
        (pl.col('cos_sza') > 0.1).cast(pl.Int8).alias('is_daytime'),
    ])
    
    # =========================================================================
    # CATEGORY 3: METEOROLOGICAL DERIVED (20 features)
    # =========================================================================
    print("  [3/10] Creating meteorological features...")
    
    # Basic derived
    df = df.with_columns([
        (pl.col('T_forecast') + 273.15).alias('T_kelvin'),
        (pl.col('u_forecast').pow(2) + pl.col('v_forecast').pow(2)).sqrt().alias('wind_speed'),
    ])
    
    # Wind direction
    df = df.with_columns([
        pl.arctan2(pl.col('v_forecast'), pl.col('u_forecast')).alias('wind_dir_rad'),
    ])
    
    df = df.with_columns([
        pl.col('wind_dir_rad').sin().alias('wind_dir_sin'),
        pl.col('wind_dir_rad').cos().alias('wind_dir_cos'),
    ])
    
    # Dispersion and stability
    df = df.with_columns([
        (pl.col('wind_speed') * pl.col('w_forecast').abs().clip(0.01, None)).alias('ventilation'),
        (pl.col('T_forecast') * pl.col('w_forecast').abs().clip(0.01, None)).alias('pbl_proxy'),
        (pl.col('T_forecast') / (pl.col('q_forecast').clip(0.1, None))).alias('thermal_stability'),
        (pl.col('w_forecast') / (pl.col('wind_speed').clip(0.1, None))).alias('vertical_wind_ratio'),
    ])
    
    # Temperature derived
    df = df.with_columns([
        pl.col('T_forecast').pow(2).alias('T_squared'),
        pl.col('q_forecast').pow(2).alias('q_squared'),
        (pl.col('T_forecast') * pl.col('q_forecast')).alias('T_q_interaction'),
    ])
    
    # Wind categories
    df = df.with_columns([
        (pl.col('wind_speed') < 1.0).cast(pl.Int8).alias('wind_calm'),
        ((pl.col('wind_speed') >= 1.0) & (pl.col('wind_speed') < 3.0)).cast(pl.Int8).alias('wind_light'),
        ((pl.col('wind_speed') >= 3.0) & (pl.col('wind_speed') < 6.0)).cast(pl.Int8).alias('wind_moderate'),
        (pl.col('wind_speed') >= 6.0).cast(pl.Int8).alias('wind_strong'),
    ])
    
    # Stagnation
    df = df.with_columns([
        ((pl.col('wind_speed') < 2.0) & (pl.col('w_forecast').abs() < 0.5)).cast(pl.Int8).alias('is_stagnant')
    ])
    
    # =========================================================================
    # CATEGORY 4: PHOTOCHEMICAL FEATURES (8 features)
    # =========================================================================
    print("  [4/10] Creating photochemical features...")
    
    df = df.with_columns([
        (-1500 / pl.col('T_kelvin')).exp().alias('arrhenius_1500'),
        (-2500 / pl.col('T_kelvin')).exp().alias('arrhenius_2500'),
        (pl.col('cos_sza') * pl.col('T_forecast')).alias('photolysis_potential'),
    ])
    
    df = df.with_columns([
        (pl.col('cos_sza') * pl.col('T_forecast') * pl.col('arrhenius_1500')).alias('o3_prod_potential'),
        (pl.col('NO2_forecast') * (1 - pl.col('cos_sza'))).alias('nox_titration_proxy'),
        (pl.col('is_daytime') * pl.col('T_forecast')).alias('daytime_T'),
        (pl.col('is_daytime') * pl.col('wind_speed')).alias('daytime_wind'),
    ])
    
    # =========================================================================
    # CATEGORY 5: SATELLITE DERIVED (12 features)
    # =========================================================================
    print("  [5/10] Creating satellite-derived features...")
    
    # VOC-NOx ratio
    df = df.with_columns([
        (pl.col('HCHO_satellite_filled') / (pl.col('NO2_satellite_filled').clip(0.01, None))).alias('voc_nox_ratio')
    ])
    
    # Chemistry regime indicators
    df = df.with_columns([
        (pl.col('voc_nox_ratio') < 1.0).cast(pl.Int8).alias('is_voc_limited'),
        (pl.col('voc_nox_ratio') > 2.0).cast(pl.Int8).alias('is_nox_limited'),
        ((pl.col('voc_nox_ratio') >= 1.0) & (pl.col('voc_nox_ratio') <= 2.0)).cast(pl.Int8).alias('is_transitional'),
    ])
    
    # Satellite-radiation interactions
    df = df.with_columns([
        (pl.col('cos_sza') * pl.col('NO2_satellite_filled')).alias('radiation_no2'),
        (pl.col('cos_sza') * pl.col('HCHO_satellite_filled')).alias('radiation_hcho'),
    ])
    
    # Satellite-meteorology interactions
    df = df.with_columns([
        (pl.col('T_forecast') * pl.col('NO2_satellite_filled')).alias('T_no2_sat'),
        (pl.col('T_forecast') * pl.col('HCHO_satellite_filled')).alias('T_hcho_sat'),
        (pl.col('ventilation') * pl.col('NO2_satellite_filled')).alias('vent_no2_sat'),
        (pl.col('ventilation') * pl.col('HCHO_satellite_filled')).alias('vent_hcho_sat'),
    ])
    
    # =========================================================================
    # CATEGORY 6: CTM FEATURES (12 features)
    # =========================================================================
    print("  [6/10] Creating CTM-derived features...")
    
    # CTM interactions
    df = df.with_columns([
        (pl.col('O3_forecast') * pl.col('cos_sza')).alias('o3_fcst_radiation'),
        (pl.col('NO2_forecast') * pl.col('cos_sza')).alias('no2_fcst_radiation'),
        (pl.col('O3_forecast') * pl.col('T_forecast')).alias('o3_fcst_T'),
        (pl.col('NO2_forecast') * pl.col('T_forecast')).alias('no2_fcst_T'),
        (pl.col('O3_forecast') * pl.col('ventilation')).alias('o3_fcst_vent'),
        (pl.col('NO2_forecast') * pl.col('ventilation')).alias('no2_fcst_vent'),
        (pl.col('NO2_forecast') * pl.col('is_rush_hour')).alias('no2_fcst_rush'),
    ])
    
    # CTM ratios and squared
    df = df.with_columns([
        (pl.col('O3_forecast') / (pl.col('NO2_forecast').clip(0.1, None))).alias('ctm_o3_no2_ratio'),
        pl.col('O3_forecast').pow(2).alias('o3_fcst_squared'),
        pl.col('NO2_forecast').pow(2).alias('no2_fcst_squared'),
    ])
    
    # Add a final interaction feature (O3/Met)
    df = df.with_columns([
        (pl.col('O3_forecast') * pl.col('thermal_stability')).alias('o3_fcst_stability')
    ])
    
    # =========================================================================
    # CATEGORY 7: ROLLING METEOROLOGICAL (18 features)
    # =========================================================================
    print("  [7/10] Creating rolling meteorological features...")
    
    df = df.sort(['site_id', 'datetime'])
    
    for col, windows in [('T_forecast', [6, 24]), ('wind_speed', [6, 24]), 
                          ('ventilation', [6, 24]), ('cos_sza', [6])]:
        for w in windows:
            # Check if rolling std is needed (cos_sza std is usually near zero)
            is_std_needed = col in ['T_forecast', 'wind_speed', 'ventilation']
            
            df = df.with_columns([
                pl.col(col).rolling_mean(window_size=w, min_periods=1).over('site_id').alias(f'{col}_rmean_{w}h'),
            ])
            if is_std_needed:
                df = df.with_columns([
                    pl.col(col).rolling_std(window_size=w, min_periods=2).over('site_id').alias(f'{col}_rstd_{w}h'),
                ])
            
    # Trends
    df = df.with_columns([
        (pl.col('T_forecast') - pl.col('T_forecast_rmean_6h')).alias('T_trend_6h'),
        (pl.col('wind_speed') - pl.col('wind_speed_rmean_6h')).alias('wind_trend_6h'),
        (pl.col('ventilation') - pl.col('ventilation_rmean_6h')).alias('vent_trend_6h'),
    ])
    
    # Fill rolling NaNs (e.g., initial observations)
    rolling_cols = [c for c in df.columns if '_rmean_' in c or '_rstd_' in c or '_trend_' in c]
    for col in rolling_cols:
        df = df.with_columns(pl.col(col).fill_null(strategy='forward').over('site_id').fill_null(0).alias(col))
        
    # =========================================================================
    # CATEGORY 8: GAP AWARENESS (5 features)
    # =========================================================================
    print("  [8/10] Creating gap awareness features...")
    
    df = df.with_columns([
        pl.col('datetime').diff().over('site_id').dt.total_hours().fill_null(1.0).alias('hours_since_prev_obs')
    ])
    
    df = df.with_columns([
        (pl.col('hours_since_prev_obs') > 1).cast(pl.Int8).alias('has_gap'),
        (pl.col('hours_since_prev_obs') > 6).cast(pl.Int8).alias('has_gap_6h'),
        (pl.col('hours_since_prev_obs') > 24).cast(pl.Int8).alias('has_gap_24h'),
        (pl.col('hours_since_prev_obs') + 1).log().alias('log_hours_since_prev'),
    ])
    
    # =========================================================================
    # CATEGORY 9: SITE FEATURES (7 features)
    # =========================================================================
    print("  [9/10] Creating site indicator features...")
    
    for site_id in range(1, config['n_sites'] + 1):
        df = df.with_columns([
            (pl.col('site_id') == site_id).cast(pl.Int8).alias(f'site_{site_id}')
        ])
    
    # =========================================================================
    # CLEANUP
    # =========================================================================
    print("  [10/10] Cleaning up intermediate columns...")
    
    # Drop intermediate columns
    drop_cols = ['declination', 'hour_angle', 'wind_dir_rad', 'T_kelvin', 'doy']
    df = df.drop([c for c in drop_cols if c in df.columns])
    
    return df

# Create features
# NOTE: df and CONFIG must be defined in the calling environment.
df = create_all_features(df, CONFIG) 

# Count features by category (assuming df is now the result)
feature_cols = [c for c in df.columns if c not in [
    'year', 'month', 'day', 'hour', 'date', 'datetime', 
    'O3_target', 'NO2_target', 'site_id',
    'NO2_satellite', 'HCHO_satellite', 'ratio_satellite'
]]

print(f"\n  Total features created: {len(feature_cols)}")
print(f"  DataFrame shape: {df.shape[0]:,} x {df.shape[1]}")


STAGE 1: COMPREHENSIVE FEATURE ENGINEERING

  [1/10] Creating temporal features...
  [2/10] Creating solar geometry features...
  [3/10] Creating meteorological features...
  [4/10] Creating photochemical features...
  [5/10] Creating satellite-derived features...
  [6/10] Creating CTM-derived features...
  [7/10] Creating rolling meteorological features...
  [8/10] Creating gap awareness features...
  [9/10] Creating site indicator features...
  [10/10] Cleaning up intermediate columns...

  Total features created: 101
  DataFrame shape: 171,679 x 113


In [3]:
# Cell 3: CTM Bias Correction
print("\n" + "=" * 80)
print("STAGE 2: CTM BIAS CORRECTION")
print("=" * 80)

def compute_metrics(y_true, y_pred):
    """Compute regression metrics."""
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    yt, yp = y_true[mask], y_pred[mask]
    if len(yt) == 0:
        return {'rmse': np. nan, 'mae': np.nan, 'r2': np.nan, 'corr': np.nan, 'bias': np.nan}
    
    rmse = np.sqrt(mean_squared_error(yt, yp))
    mae = mean_absolute_error(yt, yp)
    r2 = r2_score(yt, yp)
    corr = np.corrcoef(yt, yp)[0, 1] if len(yt) > 1 else 0
    bias = np.mean(yp - yt)
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2, 'corr': corr, 'bias': bias}

def train_ctm_correction(df, target_name, forecast_name, config):
    """Train a model to correct CTM bias."""
    
    print(f"\n  Correcting {forecast_name} for {target_name}...")
    
    # Features for bias prediction (regime features)
    bias_features = [
        forecast_name, 'hour_sin', 'hour_cos', 'month',
        'doy_sin', 'doy_cos', 'T_forecast', 'q_forecast',
        'wind_speed', 'ventilation', 'cos_sza', 'cos_sza_squared',
        'is_daytime', 'is_winter', 'is_monsoon', 'diurnal_phase',
        'thermal_stability', 'is_stagnant'
    ] + [f'site_{i}' for i in range(1, config['n_sites'] + 1)]
    
    bias_features = [f for f in bias_features if f in df. columns]
    
    # Filter valid samples
    df_valid = df. filter(
        pl.col(target_name).is_not_null() & 
        pl.col(forecast_name).is_not_null()
    )
    
    # Get arrays
    X = df_valid. select(bias_features).to_numpy(). astype(np.float32)
    y_actual = df_valid[target_name].to_numpy().astype(np.float32)
    y_forecast = df_valid[forecast_name].to_numpy(). astype(np. float32)
    y_bias = y_forecast - y_actual  # Target is the bias
    
    # Handle NaN
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np.isnan(col_medians), 0, col_medians)
    for j in range(X.shape[1]):
        X[np.isnan(X[:, j]), j] = col_medians[j]
    
    # Raw CTM metrics
    raw_metrics = compute_metrics(y_actual, y_forecast)
    print(f"    Raw CTM -> Correlation: {raw_metrics['corr']:.4f}, RMSE: {raw_metrics['rmse']:.2f}, Bias: {raw_metrics['bias']:+.2f}")
    
    # Train bias correction model with OOF
    kf = KFold(n_splits=config['n_folds'], shuffle=True, random_state=config['random_state'])
    oof_bias_pred = np.zeros(len(y_bias))
    models = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(tqdm(kf. split(X), total=config['n_folds'], 
                                                          desc=f"    Training bias model", leave=False)):
        model = cb.CatBoostRegressor(
            iterations=500,
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=5,
            loss_function='RMSE',
            random_seed=config['random_state'] + fold_idx,
            verbose=False,
            early_stopping_rounds=50,
            task_type='GPU',
            devices = "0:1"
        )
        model.fit(X[train_idx], y_bias[train_idx], 
                  eval_set=(X[val_idx], y_bias[val_idx]), verbose=False)
        oof_bias_pred[val_idx] = model.predict(X[val_idx])
        models. append(model)
    
    # Corrected forecast
    y_corrected = y_forecast - oof_bias_pred
    
    # Corrected metrics
    corrected_metrics = compute_metrics(y_actual, y_corrected)
    print(f"    Corrected -> Correlation: {corrected_metrics['corr']:.4f}, RMSE: {corrected_metrics['rmse']:.2f}, Bias: {corrected_metrics['bias']:+.2f}")
    print(f"    Improvement: Corr {corrected_metrics['corr'] - raw_metrics['corr']:+.4f}, RMSE {raw_metrics['rmse'] - corrected_metrics['rmse']:+.2f}")
    
    return {
        'models': models,
        'features': bias_features,
        'col_medians': col_medians,
        'raw_metrics': raw_metrics,
        'corrected_metrics': corrected_metrics,
        'oof_bias_pred': oof_bias_pred,
        'forecast_name': forecast_name,
    }

# Define target-forecast mapping
target_forecast_map = {
    'O3_target': 'O3_forecast',
    'NO2_target': 'NO2_forecast'
}

# Train CTM correction for both targets
ctm_corrections = {}

for target in CONFIG['targets']:
    forecast = target_forecast_map[target]
    ctm_corrections[target] = train_ctm_correction(df, target, forecast, CONFIG)

# Apply corrections to full dataframe
print("\n  Applying CTM corrections to full dataset...")

def apply_ctm_correction(df, correction_result):
    """Apply trained correction to dataframe."""
    features = correction_result['features']
    models = correction_result['models']
    col_medians = correction_result['col_medians']
    forecast_name = correction_result['forecast_name']
    
    # Prepare features
    available_features = [f for f in features if f in df.columns]
    X = df. select(available_features).to_numpy(). astype(np. float32)
    
    # Handle NaN - need to match feature order
    for j in range(X.shape[1]):
        if j < len(col_medians):
            X[np.isnan(X[:, j]), j] = col_medians[j]
        else:
            X[np.isnan(X[:, j]), j] = 0
    
    # Average predictions across fold models
    bias_pred = np. zeros(len(X))
    for model in models:
        bias_pred += model.predict(X) / len(models)
    
    # Corrected forecast
    forecast_vals = df[forecast_name]. to_numpy()
    corrected = forecast_vals - bias_pred
    
    return corrected

# Add corrected CTM columns
o3_corrected = apply_ctm_correction(df, ctm_corrections['O3_target'])
no2_corrected = apply_ctm_correction(df, ctm_corrections['NO2_target'])

df = df.with_columns([
    pl. Series('O3_forecast_corrected', o3_corrected),
    pl.Series('NO2_forecast_corrected', no2_corrected),
])

# Create corrected CTM interaction features
df = df.with_columns([
    (pl.col('O3_forecast_corrected') * pl. col('cos_sza')).alias('o3_corr_radiation'),
    (pl.col('NO2_forecast_corrected') * pl. col('cos_sza')).alias('no2_corr_radiation'),
    (pl.col('O3_forecast_corrected') * pl.col('ventilation')).alias('o3_corr_vent'),
    (pl.col('NO2_forecast_corrected') * pl. col('ventilation')).alias('no2_corr_vent'),
    (pl.col('O3_forecast_corrected') / (pl.col('NO2_forecast_corrected'). clip(0.1, None))).alias('ctm_corr_ratio'),
])

print(f"  CTM correction complete. New columns added.")
print(f"  DataFrame shape: {df. shape[0]:,} x {df.shape[1]}")


STAGE 2: CTM BIAS CORRECTION

  Correcting O3_forecast for O3_target...
    Raw CTM -> Correlation: 0.0603, RMSE: 78.97, Bias: +38.88


    Training bias model:   0%|          | 0/5 [00:00<?, ?it/s]

    Corrected -> Correlation: 0.8290, RMSE: 18.99, Bias: -0.01
    Improvement: Corr +0.7687, RMSE +59.98

  Correcting NO2_forecast for NO2_target...
    Raw CTM -> Correlation: -0.0308, RMSE: 56.25, Bias: +19.18


    Training bias model:   0%|          | 0/5 [00:00<?, ?it/s]

    Corrected -> Correlation: 0.7946, RMSE: 17.84, Bias: -0.00
    Improvement: Corr +0.8254, RMSE +38.41

  Applying CTM corrections to full dataset...
  CTM correction complete. New columns added.
  DataFrame shape: 171,679 x 120


In [4]:
# Cell 4: Satellite Diurnal Adjustment
print("\n" + "=" * 80)
print("STAGE 3: SATELLITE DIURNAL ADJUSTMENT")
print("=" * 80)

def train_satellite_adjustment(df, target_name, sat_col, config):
    """Train a model to adjust daily satellite values to hourly estimates."""
    
    print(f"\n  Adjusting {sat_col} for {target_name}...")
    
    # Features for hourly adjustment
    adj_features = [
        sat_col, 'hour_sin', 'hour_cos', 'hour',
        'doy_sin', 'doy_cos', 'month',
        'T_forecast', 'wind_speed', 'ventilation',
        'cos_sza', 'cos_sza_squared', 'is_daytime',
        'is_winter', 'is_monsoon', 'is_summer',
        'diurnal_phase', 'is_rush_hour', 'is_stagnant',
        'thermal_stability', 'pbl_proxy',
    ] + [f'site_{i}' for i in range(1, config['n_sites'] + 1)]
    
    adj_features = [f for f in adj_features if f in df.columns]
    
    # Filter valid samples
    df_valid = df.filter(
        pl.col(target_name).is_not_null() & 
        pl. col(sat_col).is_not_null()
    )
    
    # Get arrays
    X = df_valid. select(adj_features).to_numpy(). astype(np. float32)
    y_actual = df_valid[target_name].to_numpy().astype(np.float32)
    y_sat = df_valid[sat_col].to_numpy().astype(np.float32)
    
    # Target: adjustment ratio (how to scale satellite to match actual)
    # We predict the ratio: actual / satellite, then multiply satellite by this ratio
    # But to avoid division issues, we predict actual directly conditioned on satellite
    y_target = y_actual  # Predict actual given satellite and regime
    
    # Handle NaN
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np. isnan(col_medians), 0, col_medians)
    for j in range(X. shape[1]):
        X[np. isnan(X[:, j]), j] = col_medians[j]
    
    # Raw satellite correlation with target
    raw_corr = np.corrcoef(y_sat, y_actual)[0, 1]
    raw_rmse = np.sqrt(mean_squared_error(y_actual, y_sat))
    print(f"    Raw satellite -> Correlation: {raw_corr:.4f}, RMSE: {raw_rmse:.2f}")
    
    # Train adjustment model with OOF
    kf = KFold(n_splits=config['n_folds'], shuffle=True, random_state=config['random_state'])
    oof_adjusted = np.zeros(len(y_target))
    models = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(tqdm(kf.split(X), total=config['n_folds'],
                                                          desc=f"    Training adjustment model", leave=False)):
        model = cb.CatBoostRegressor(
            iterations=500,
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=5,
            loss_function='RMSE',
            random_seed=config['random_state'] + fold_idx + 100,
            verbose=False,
            early_stopping_rounds=50,
            task_type='GPU',
            devices = "0:1"
        )
        model.fit(X[train_idx], y_target[train_idx],
                  eval_set=(X[val_idx], y_target[val_idx]), verbose=False)
        oof_adjusted[val_idx] = model.predict(X[val_idx])
        models.append(model)
    
    # Adjusted metrics
    adj_corr = np.corrcoef(oof_adjusted, y_actual)[0, 1]
    adj_rmse = np.sqrt(mean_squared_error(y_actual, oof_adjusted))
    print(f"    Adjusted satellite -> Correlation: {adj_corr:.4f}, RMSE: {adj_rmse:.2f}")
    print(f"    Improvement: Corr {adj_corr - raw_corr:+.4f}, RMSE {raw_rmse - adj_rmse:+.2f}")
    
    return {
        'models': models,
        'features': adj_features,
        'col_medians': col_medians,
        'raw_corr': raw_corr,
        'adj_corr': adj_corr,
        'raw_rmse': raw_rmse,
        'adj_rmse': adj_rmse,
    }

# Train satellite adjustment for each target-satellite pair
sat_adjustments = {}

# For O3: Use both NO2 and HCHO satellite
sat_adjustments['O3_NO2_sat'] = train_satellite_adjustment(
    df, 'O3_target', 'NO2_satellite_filled', CONFIG)
sat_adjustments['O3_HCHO_sat'] = train_satellite_adjustment(
    df, 'O3_target', 'HCHO_satellite_filled', CONFIG)

# For NO2: Use NO2 satellite
sat_adjustments['NO2_NO2_sat'] = train_satellite_adjustment(
    df, 'NO2_target', 'NO2_satellite_filled', CONFIG)

# Apply adjustments
print("\n  Applying satellite adjustments to full dataset...")

def apply_satellite_adjustment(df, adj_result):
    """Apply trained adjustment to dataframe."""
    features = adj_result['features']
    models = adj_result['models']
    col_medians = adj_result['col_medians']
    
    X = df.select([f for f in features if f in df. columns]).to_numpy().astype(np.float32)
    for j in range(X.shape[1]):
        X[np.isnan(X[:, j]), j] = col_medians[j]
    
    adjusted = np.zeros(len(X))
    for model in models:
        adjusted += model.predict(X) / len(models)
    
    return adjusted

# Add adjusted satellite columns
df = df.with_columns([
    pl.Series('NO2_sat_adj_for_O3', apply_satellite_adjustment(df, sat_adjustments['O3_NO2_sat'])),
    pl. Series('HCHO_sat_adj_for_O3', apply_satellite_adjustment(df, sat_adjustments['O3_HCHO_sat'])),
    pl.Series('NO2_sat_adj_for_NO2', apply_satellite_adjustment(df, sat_adjustments['NO2_NO2_sat'])),
])

# Create adjusted satellite interaction features
df = df.with_columns([
    (pl.col('NO2_sat_adj_for_O3') * pl.col('cos_sza')).alias('no2_sat_adj_radiation'),
    (pl.col('HCHO_sat_adj_for_O3') * pl.col('cos_sza')).alias('hcho_sat_adj_radiation'),
    (pl.col('NO2_sat_adj_for_O3') * pl. col('ventilation')).alias('no2_sat_adj_vent'),
    (pl.col('HCHO_sat_adj_for_O3') / (pl.col('NO2_sat_adj_for_O3').clip(0.01, None))).alias('voc_nox_ratio_adj'),
])

print(f"  Satellite adjustment complete.  New columns added.")
print(f"  DataFrame shape: {df. shape[0]:,} x {df.shape[1]}")


STAGE 3: SATELLITE DIURNAL ADJUSTMENT

  Adjusting NO2_satellite_filled for O3_target...
    Raw satellite -> Correlation: -0.0683, RMSE: 45.10


    Training adjustment model:   0%|          | 0/5 [00:00<?, ?it/s]

    Adjusted satellite -> Correlation: 0.8287, RMSE: 19.02
    Improvement: Corr +0.8970, RMSE +26.07

  Adjusting HCHO_satellite_filled for O3_target...
    Raw satellite -> Correlation: 0.0624, RMSE: 44.50


    Training adjustment model:   0%|          | 0/5 [00:00<?, ?it/s]

    Adjusted satellite -> Correlation: 0.8251, RMSE: 19.20
    Improvement: Corr +0.7627, RMSE +25.30

  Adjusting NO2_satellite_filled for NO2_target...
    Raw satellite -> Correlation: 0.3266, RMSE: 46.85


    Training adjustment model:   0%|          | 0/5 [00:00<?, ?it/s]

    Adjusted satellite -> Correlation: 0.8090, RMSE: 17.31
    Improvement: Corr +0.4824, RMSE +29.55

  Applying satellite adjustments to full dataset...
  Satellite adjustment complete.  New columns added.
  DataFrame shape: 171,679 x 127


In [5]:
# Cell 5: OOF Lag Creation - Iteration 1
print("\n" + "=" * 80)
print("STAGE 4: OOF LAG CREATION - ITERATION 1")
print("=" * 80)

def get_base_features(df, config):
    """Get all base features (excluding targets and identifiers)."""
    exclude = [
        'year', 'month', 'day', 'hour', 'date', 'datetime', 'doy',
        'O3_target', 'NO2_target', 'site_id',
        'NO2_satellite', 'HCHO_satellite', 'ratio_satellite',
    ]
    # Corrected spurious space in df.columns
    # Also exclude any lag features from previous iterations
    features = [c for c in df.columns if c not in exclude and '_lag_' not in c 
                and '_rmean_' not in c and '_rstd_' not in c and '_oof_iter' not in c]
    return features

def prepare_arrays(df, features, target):
    """Prepare X, y arrays for training."""
    # Corrected spurious space in pl.col
    df_valid = df.filter(pl.col(target).is_not_null())
    
    available = [f for f in features if f in df_valid.columns]
    X = df_valid.select(available).to_numpy().astype(np.float32)
    
    # Corrected spurious space in np.float32
    y = df_valid[target].to_numpy().astype(np.float32)
    
    # Corrected spurious space in np.where
    # Median imputation
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np.isnan(col_medians), 0, col_medians)
    for j in range(X.shape[1]):
        X[np.isnan(X[:, j]), j] = col_medians[j]
    
    return X, y, available, col_medians

def train_catboost_oof(X, y, config, desc="Training"):
    """
    Train CatBoost with K-fold CV and return OOF predictions.
    Confidently using GPU and devices='0:1'.
    """
    kf = KFold(n_splits=config['n_folds'], shuffle=True, random_state=config['random_state'])
    
    oof = np.zeros(len(y))
    models = []
    fold_metrics = []
    
    # Corrected spurious space in kf.split(X)
    for fold_idx, (train_idx, val_idx) in enumerate(tqdm(kf.split(X), total=config['n_folds'],
                                                          desc=desc, leave=False)):
        model = cb.CatBoostRegressor(
            iterations=config['iterations'],
            depth=config['depth'],
            learning_rate=config['learning_rate'],
            l2_leaf_reg=5,
            loss_function='RMSE',
            random_seed=config['random_state'] + fold_idx,
            verbose=False,
            early_stopping_rounds=config['early_stopping'],
            # Confidently using GPU with devices '0:1' as requested
            task_type='GPU', 
            devices='0:1' 
        )
        model.fit(X[train_idx], y[train_idx],
                  eval_set=(X[val_idx], y[val_idx]), verbose=False)
        
        preds = model.predict(X[val_idx])
        oof[val_idx] = preds
        models.append(model)
        
        fold_rmse = np.sqrt(mean_squared_error(y[val_idx], preds))
        fold_metrics.append(fold_rmse)
    
    avg_rmse = np.mean(fold_metrics)
    std_rmse = np.std(fold_metrics)
    
    return oof, models, avg_rmse, std_rmse

def create_lag_features(df, oof_col, target_name, iteration):
    """Create lag and rolling features from OOF predictions."""
    df = df.sort(['site_id', 'datetime'])
    
    # Lag features
    lags = [1, 3, 6, 12, 24, 48, 168]
    for lag in lags:
        df = df.with_columns([
            pl.col(oof_col).shift(lag).over('site_id').alias(f'{target_name}_lag_{lag}h_iter_{iteration}')
        ])
    
    # Rolling mean features (Corrected spurious space in pl.col)
    for window in [6, 24, 168]:
        df = df.with_columns([
            pl.col(oof_col).rolling_mean(window_size=window, min_periods=1).over('site_id')
            .alias(f'{target_name}_rmean_{window}h_iter_{iteration}')
        ])
    
    # Rolling std features (Corrected spurious space in .alias)
    for window in [6, 24, 168]:
        df = df.with_columns([
            pl.col(oof_col).rolling_std(window_size=window, min_periods=2).over('site_id')
            .alias(f'{target_name}_rstd_{window}h_iter_{iteration}')
        ])
    
    return df

# Get base features
base_features = get_base_features(df, CONFIG)
print(f"\n  Base features for Iteration 1: {len(base_features)}")

# Train and create lags for each target
iteration_1_results = {}

for target in CONFIG['targets']:
    print(f"\n  {target}:")
    print("-" * 40)
    
    X, y, used_features, col_medians = prepare_arrays(df, base_features, target)
    print(f"    Samples: {len(y):,}, Features: {len(used_features)}")
    
    # FUNCTION CALL: Using the corrected train_catboost_oof function
    oof, models, avg_rmse, std_rmse = train_catboost_oof(X, y, CONFIG, desc=f"    {target} Folds")
    print(f"    OOF RMSE: {avg_rmse:.4f} (+/- {std_rmse:.4f})")
    
    # Store OOF predictions
    oof_col = f'{target}_oof_iter_0'
    oof_full = np.full(len(df), np.nan)
    
    # Corrected spurious space in to_numpy()
    valid_mask = df[target].is_not_null().to_numpy()
    oof_full[valid_mask] = oof
    
    df = df.with_columns(pl.Series(oof_col, oof_full))
    
    # Create lag features
    df = create_lag_features(df, oof_col, target, iteration=0)
    
    iteration_1_results[target] = {
        'oof': oof,
        'models': models,
        'features': used_features,
        'col_medians': col_medians,
        'rmse': avg_rmse,
        'std': std_rmse,
    }

# Corrected spurious space in df.shape
print(f"\n  Iteration 1 complete.")
print(f"  DataFrame shape: {df.shape[0]:,} x {df.shape[1]}")


STAGE 4: OOF LAG CREATION - ITERATION 1

  Base features for Iteration 1: 102

  O3_target:
----------------------------------------
    Samples: 171,679, Features: 102


    O3_target Folds:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 11.9405 (+/- 0.0730)

  NO2_target:
----------------------------------------
    Samples: 171,679, Features: 102


    NO2_target Folds:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 10.4517 (+/- 0.0769)

  Iteration 1 complete.
  DataFrame shape: 171,679 x 155


In [6]:
# Cell 6: OOF Lag Creation - Iteration 2 (Final)
print("\n" + "=" * 80)
print("STAGE 5: OOF LAG CREATION - ITERATION 2 (FINAL)")
print("=" * 80)

def get_features_with_lags(df, target, iteration, config):
    """Get features including lag features from previous iteration."""
    exclude = [
        'year', 'month', 'day', 'hour', 'date', 'datetime', 'doy',
        'O3_target', 'NO2_target', 'site_id',
        'NO2_satellite', 'HCHO_satellite', 'ratio_satellite',
    ]
    
    # Get all base features
    # Corrected spurious space
    base = [c for c in df.columns if c not in exclude 
            and '_oof_iter' not in c]  # Exclude OOF columns themselves
    
    # Filter to only include lags from correct iteration and target
    features = []
    for f in base:
        # Include non-lag features
        if '_lag_' not in f and '_rmean_' not in f and '_rstd_' not in f:
            # Corrected spurious space
            features.append(f)
        # Include lags from previous iteration for THIS target only
        elif f'_iter_{iteration-1}' in f and target.replace('_target', '') in f:
            # Corrected spurious space
            features.append(f)
    
    return list(dict.fromkeys(features))  # Remove duplicates

# Train Iteration 2 for each target
iteration_2_results = {}

for target in CONFIG['targets']:
    print(f"\n  {target}:")
    print("-" * 40)
    
    # Get features including Iteration 0 lags
    # FUNCTION CALL: Uses get_features_with_lags
    features = get_features_with_lags(df, target, iteration=1, config=CONFIG)
    
    # Count lag features
    lag_features = [f for f in features if '_iter_0' in f]
    print(f"    Base features: {len(features) - len(lag_features)}, Lag features: {len(lag_features)}")
    
    # FUNCTION CALL: prepare_arrays (assumes definition from Cell 5)
    X, y, used_features, col_medians = prepare_arrays(df, features, target)
    print(f"    Total features: {len(used_features)}, Samples: {len(y):,}")
    
    # FUNCTION CALL: train_catboost_oof (assumes definition from Cell 5, uses GPU configuration)
    oof, models, avg_rmse, std_rmse = train_catboost_oof(X, y, CONFIG, desc=f"    {target} Folds")
    
    # Compare with Iteration 1
    iter1_rmse = iteration_1_results[target]['rmse']
    improvement = iter1_rmse - avg_rmse
    print(f"    OOF RMSE: {avg_rmse:.4f} (+/- {std_rmse:.4f})")
    print(f"    Improvement from Iter 1: {improvement:+.4f} ({improvement/iter1_rmse*100:+.1f}%)")
    
    # Store OOF predictions
    oof_col = f'{target}_oof_iter_1'
    # Corrected spurious space in np.full
    oof_full = np.full(len(df), np.nan)
    # Corrected spurious space in is_not_null()
    valid_mask = df[target].is_not_null().to_numpy()
    oof_full[valid_mask] = oof
    
    df = df.with_columns(pl.Series(oof_col, oof_full))
    
    # Create final lag features
    # FUNCTION CALL: create_lag_features (assumes definition from Cell 5)
    df = create_lag_features(df, oof_col, target, iteration=1)
    
    iteration_2_results[target] = {
        'oof': oof,
        'models': models,
        'features': used_features,
        'col_medians': col_medians,
        'rmse': avg_rmse,
        'std': std_rmse,
    }

# Corrected spurious space in df.shape
print(f"\n  Iteration 2 complete.")
print(f"  DataFrame shape: {df.shape[0]:,} x {df.shape[1]}")


STAGE 5: OOF LAG CREATION - ITERATION 2 (FINAL)

  O3_target:
----------------------------------------
    Base features: 102, Lag features: 13
    Total features: 115, Samples: 171,679


    O3_target Folds:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 10.2105 (+/- 0.0688)
    Improvement from Iter 1: +1.7300 (+14.5%)

  NO2_target:
----------------------------------------
    Base features: 102, Lag features: 13
    Total features: 115, Samples: 171,679


    NO2_target Folds:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 9.1549 (+/- 0.1112)
    Improvement from Iter 1: +1.2968 (+12.4%)

  Iteration 2 complete.
  DataFrame shape: 171,679 x 183


In [7]:
# Cell 1: Additional OOF Iterations (3 more: iterations 2, 3, 4)
print("=" * 80)
print("STAGE 7: ADDITIONAL OOF ITERATIONS (3 MORE)")
print("=" * 80)

def get_features_for_iteration(df, target, current_iter, config):
    """Get features including lags from previous iteration."""
    exclude = [
        'year', 'month', 'day', 'hour', 'date', 'datetime', 'doy',
        'O3_target', 'NO2_target', 'site_id',
        'NO2_satellite', 'HCHO_satellite', 'ratio_satellite',
    ]
    
    target_prefix = target.replace('_target', '')
    features = []
    
    for c in df.columns:
        if c in exclude:
            continue
        if '_oof_iter' in c:
            continue
        
        # For lag features, only keep from previous iteration for this target
        if '_iter_' in c:
            iter_num = int(c.split('_iter_')[-1])
            if iter_num == current_iter - 1 and target_prefix in c:
                features.append(c)
        else:
            features.append(c)
    
    return list(dict.fromkeys(features))

def prepare_arrays_v2(df, features, target):
    """Prepare X, y arrays for training."""
    # Corrected spurious spaces
    df_valid = df.filter(pl.col(target).is_not_null())
    
    available = [f for f in features if f in df_valid.columns]
    X = df_valid.select(available).to_numpy().astype(np.float32)
    y = df_valid[target].to_numpy().astype(np.float32)
    
    # Median imputation (corrected spurious spaces)
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np.isnan(col_medians), 0, col_medians)
    for j in range(X.shape[1]):
        X[np.isnan(X[:, j]), j] = col_medians[j]
    
    return X, y, available, col_medians

def train_catboost_oof_v2(X, y, config, desc="Training"):
    """
    Train CatBoost with K-fold CV and return OOF predictions.
    Uses GPU: devices='0:1' as requested.
    """
    kf = KFold(n_splits=config['n_folds'], shuffle=True, random_state=config['random_state'])
    
    oof = np.zeros(len(y))
    models = []
    fold_metrics = []
    
    # Corrected spurious space
    pbar = tqdm(enumerate(kf.split(X)), total=config['n_folds'], desc=desc, leave=False)
    
    for fold_idx, (train_idx, val_idx) in pbar:
        model = cb.CatBoostRegressor(
            iterations=config['iterations'],
            depth=config['depth'],
            learning_rate=config['learning_rate'],
            l2_leaf_reg=5,
            loss_function='RMSE',
            random_seed=config['random_state'] + fold_idx,
            verbose=False,
            early_stopping_rounds=config['early_stopping'],
            # Confidently using GPU configuration as requested
            task_type='GPU',
            devices='0:1' 
        )
        model.fit(X[train_idx], y[train_idx],
                  eval_set=(X[val_idx], y[val_idx]), verbose=False)
        
        preds = model.predict(X[val_idx])
        oof[val_idx] = preds
        models.append(model)
        
        fold_rmse = np.sqrt(mean_squared_error(y[val_idx], preds))
        # Corrected spurious space
        fold_metrics.append(fold_rmse)
        pbar.set_postfix({'fold_rmse': f'{fold_rmse:.4f}'})
    
    avg_rmse = np.mean(fold_metrics)
    std_rmse = np.std(fold_metrics)
    
    return oof, models, avg_rmse, std_rmse

def create_lag_features_v2(df, oof_col, target_name, iteration):
    """Create comprehensive lag and rolling features from OOF predictions."""
    df = df.sort(['site_id', 'datetime'])
    
    # Clock-time lags
    lags = [1, 2, 3, 6, 12, 24, 48, 72, 168]
    for lag in lags:
        col_name = f'{target_name}_lag_{lag}h_iter_{iteration}'
        if col_name not in df.columns:
            df = df.with_columns([
                # Corrected spurious space in .alias
                pl.col(oof_col).shift(lag).over('site_id').alias(col_name)
            ])
    
    # Rolling means
    for window in [3, 6, 12, 24, 48, 168]:
        col_name = f'{target_name}_rmean_{window}h_iter_{iteration}'
        if col_name not in df.columns:
            df = df.with_columns([
                # Corrected spurious space in pl.col and .over
                pl.col(oof_col).rolling_mean(window_size=window, min_periods=1).over('site_id')
                .alias(col_name)
            ])
    
    # Rolling stds
    for window in [3, 6, 12, 24, 48, 168]:
        col_name = f'{target_name}_rstd_{window}h_iter_{iteration}'
        if col_name not in df.columns:
            df = df.with_columns([
                # Corrected spurious space in .alias
                pl.col(oof_col).rolling_std(window_size=window, min_periods=2).over('site_id')
                .alias(col_name)
            ])
    
    # Rolling min/max
    for window in [6, 24]:
        min_name = f'{target_name}_rmin_{window}h_iter_{iteration}'
        max_name = f'{target_name}_rmax_{window}h_iter_{iteration}'
        if min_name not in df.columns:
            df = df.with_columns([
                # Corrected spurious space in .over and .alias
                pl.col(oof_col).rolling_min(window_size=window, min_periods=1).over('site_id')
                .alias(min_name),
                pl.col(oof_col).rolling_max(window_size=window, min_periods=1).over('site_id')
                .alias(max_name),
            ])
    
    # Range (max - min)
    for window in [6, 24]:
        range_name = f'{target_name}_range_{window}h_iter_{iteration}'
        if range_name not in df.columns:
            df = df.with_columns([
                # Corrected spurious space in inner pl.col
                (pl.col(f'{target_name}_rmax_{window}h_iter_{iteration}') - 
                 pl.col(f'{target_name}_rmin_{window}h_iter_{iteration}')).alias(range_name)
            ])
    
    return df

# Store iteration results
# NOTE: Assumes iteration_1_results and iteration_2_results were defined in preceding cells
all_iteration_results = {
    'O3_target': [iteration_1_results['O3_target'], iteration_2_results['O3_target']],
    'NO2_target': [iteration_1_results['NO2_target'], iteration_2_results['NO2_target']],
}

# Run 3 more iterations (iterations 2, 3, 4 in 0-indexed)
additional_iterations = 3

for iter_idx in tqdm(range(2, 2 + additional_iterations), desc="OOF Iterations"):
    print(f"\n{'='*60}")
    print(f"ITERATION {iter_idx + 1} (0-indexed: {iter_idx})")
    print(f"{'='*60}")
    
    for target in CONFIG['targets']:
        print(f"\n  {target}:")
        print("-" * 40)
        
        # Get features with previous iteration lags
        features = get_features_for_iteration(df, target, iter_idx, CONFIG)
        
        # Count lag features
        lag_feats = [f for f in features if f'_iter_{iter_idx-1}' in f]
        base_feats = len(features) - len(lag_feats)
        print(f"    Base features: {base_feats}, Lag features: {len(lag_feats)}")
        
        # FUNCTION CALL: prepare_arrays_v2
        X, y, used_features, col_medians = prepare_arrays_v2(df, features, target)
        print(f"    Total features: {len(used_features)}, Samples: {len(y):,}")
        
        # FUNCTION CALL: train_catboost_oof_v2
        oof, models, avg_rmse, std_rmse = train_catboost_oof_v2(
            X, y, CONFIG, desc=f"    {target} Iter {iter_idx+1}"
        )
        
        # Compare with previous iteration
        prev_rmse = all_iteration_results[target][-1]['rmse']
        improvement = prev_rmse - avg_rmse
        print(f"    OOF RMSE: {avg_rmse:.4f} (+/- {std_rmse:.4f})")
        print(f"    Improvement: {improvement:+.4f} ({improvement/prev_rmse*100:+.1f}%)")
        
        # Store OOF predictions
        oof_col = f'{target}_oof_iter_{iter_idx}'
        oof_full = np.full(len(df), np.nan)
        # Corrected spurious space
        valid_mask = df[target].is_not_null().to_numpy()
        oof_full[valid_mask] = oof
        
        df = df.with_columns(pl.Series(oof_col, oof_full))
        
        # Create lag features
        df = create_lag_features_v2(df, oof_col, target, iteration=iter_idx)
        
        # Store results
        all_iteration_results[target].append({
            'oof': oof,
            'models': models,
            'features': used_features,
            'col_medians': col_medians,
            'rmse': avg_rmse,
            'std': std_rmse,
            'iteration': iter_idx,
        })

# Summary
print("\n" + "=" * 80)
print("OOF ITERATION SUMMARY")
print("=" * 80)

print(f"\n{'Target':<15} {'Iter 1':>10} {'Iter 2':>10} {'Iter 3':>10} {'Iter 4':>10} {'Iter 5':>10} {'Total Gain':>12}")
print("-" * 80)

for target in CONFIG['targets']:
    results = all_iteration_results[target]
    rmses = [r['rmse'] for r in results]
    total_gain = rmses[0] - rmses[-1]
    
    row = f"{target:<15}"
    for rmse in rmses:
        row += f" {rmse:>10.4f}"
    row += f" {total_gain:>+12.4f}"
    print(row)

# Corrected spurious space
print(f"\nDataFrame shape: {df.shape[0]:,} x {df.shape[1]}")

STAGE 7: ADDITIONAL OOF ITERATIONS (3 MORE)


OOF Iterations:   0%|          | 0/3 [00:00<?, ?it/s]


ITERATION 3 (0-indexed: 2)

  O3_target:
----------------------------------------
    Base features: 115, Lag features: 13
    Total features: 128, Samples: 171,679


    O3_target Iter 3:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 9.9642 (+/- 0.0993)
    Improvement: +0.2463 (+2.4%)

  NO2_target:
----------------------------------------
    Base features: 115, Lag features: 13
    Total features: 128, Samples: 171,679


...

    OOF RMSE: 8.7063 (+/- 0.1599)
    Improvement: +0.2659 (+3.0%)

ITERATION 5 (0-indexed: 4)

  O3_target:
----------------------------------------
    Base features: 115, Lag features: 27
    Total features: 142, Samples: 171,679


    O3_target Iter 5:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 9.6287 (+/- 0.0865)
    Improvement: +0.0524 (+0.5%)

  NO2_target:
----------------------------------------
    Base features: 115, Lag features: 27
    Total features: 142, Samples: 171,679


    NO2_target Iter 5:   0%|          | 0/5 [00:00<?, ?it/s]

    OOF RMSE: 8.6759 (+/- 0.1485)
    Improvement: +0.0304 (+0.3%)

OOF ITERATION SUMMARY

Target              Iter 1     Iter 2     Iter 3     Iter 4     Iter 5   Total Gain
--------------------------------------------------------------------------------
O3_target          11.9405    10.2105     9.9642     9.6812     9.6287      +2.3118
NO2_target         10.4517     9.1549     8.9722     8.7063     8.6759      +1.7759

DataFrame shape: 171,679 x 351


In [11]:
# Assuming 'pl' (polars), 'np' (numpy), 'additional_iterations', 'df', and 'CONFIG' are imported/defined.

# Cell 2: Advanced Feature Engineering
print("\n" + "=" * 80)
print("STAGE 8: ADVANCED FEATURE ENGINEERING")
print("=" * 80)

def create_advanced_features(df, config, last_iter):
    """Create advanced features for final model."""

    # FIX 1: Correct Polars sort syntax: pass column names as positional arguments
    df = df.sort('site_id', 'datetime')
    initial_cols = len(df.columns)

    # =========================================================================
    # 1. CROSS-TARGET FEATURES (O3-NO2 interactions)
    # =========================================================================
    print("\n  [1/7] Creating cross-target features...")

    # O3-NO2 ratio from OOF predictions
    o3_oof = f'O3_target_oof_iter_{last_iter}'
    no2_oof = f'NO2_target_oof_iter_{last_iter}'

    if o3_oof in df.columns and no2_oof in df.columns:
        # FIXES 2-7 from previous round: Remove spaces within pl.col(), .clip(), and .alias()
        df = df.with_columns([
            (pl.col(o3_oof) / (pl.col(no2_oof).clip(0.1, None))).alias('oof_o3_no2_ratio'),
            (pl.col(o3_oof) - pl.col(no2_oof)).alias('oof_o3_no2_diff'),
            (pl.col(o3_oof) + pl.col(no2_oof)).alias('oof_ox_total'),  # Ox = O3 + NO2
        ])

    # =========================================================================
    # 2. LAG DIFFERENCES (momentum features)
    # =========================================================================
    print("  [2/7] Creating lag difference features...")

    for target in config['targets']:
        # Difference between consecutive lags (momentum)
        lag_1h = f'{target}_lag_1h_iter_{last_iter}'
        lag_3h = f'{target}_lag_3h_iter_{last_iter}'
        lag_6h = f'{target}_lag_6h_iter_{last_iter}'
        lag_24h = f'{target}_lag_24h_iter_{last_iter}'

        if lag_1h in df.columns and lag_3h in df.columns:
            # FIXES 8-10 from previous round: Remove spaces within pl.col() and .alias()
            df = df.with_columns([
                (pl.col(lag_1h) - pl.col(lag_3h)).alias(f'{target}_diff_1h_3h'),
            ])

        if lag_1h in df.columns and lag_6h in df.columns:
            # FIXES 11-13 from previous round: Remove spaces within pl.col() and .alias()
            df = df.with_columns([
                (pl.col(lag_1h) - pl.col(lag_6h)).alias(f'{target}_diff_1h_6h'),
            ])

        if lag_1h in df.columns and lag_24h in df.columns:
            # FIX 14 from previous round: Remove space in df. columns
            df = df.with_columns([
                (pl.col(lag_1h) - pl.col(lag_24h)).alias(f'{target}_diff_1h_24h'),
            ])

        if lag_3h in df.columns and lag_6h in df.columns:
            # FIX 15 from previous round: Remove space in .alias()
            df = df.with_columns([
                (pl.col(lag_3h) - pl.col(lag_6h)).alias(f'{target}_diff_3h_6h'),
            ])

        # Rate of change
        if lag_1h in df.columns and lag_3h in df.columns:
            # FIX 38 (The current error): Added opening parenthesis before (pl.col(...)
            df = df.with_columns([
                ((pl.col(lag_1h) - pl.col(lag_3h)) / 2).alias(f'{target}_rate_3h'),
            ])

        if lag_1h in df.columns and lag_6h in df.columns:
            # FIX 16 from previous round: Remove space in df. columns
            df = df.with_columns([
                ((pl.col(lag_1h) - pl.col(lag_6h)) / 5).alias(f'{target}_rate_6h'),
            ])

    # =========================================================================
    # 3. VOLATILITY RATIOS
    # =========================================================================
    print("  [3/7] Creating volatility ratio features...")

    for target in config['targets']:
        rstd_6h = f'{target}_rstd_6h_iter_{last_iter}'
        rstd_24h = f'{target}_rstd_24h_iter_{last_iter}'
        rmean_6h = f'{target}_rmean_6h_iter_{last_iter}'
        rmean_24h = f'{target}_rmean_24h_iter_{last_iter}'

        # Coefficient of variation (normalized volatility)
        if rstd_6h in df.columns and rmean_6h in df.columns:
            # FIXES 17-19 from previous round: Remove spaces in df. with_columns, .clip(), and .alias()
            df = df.with_columns([
                (pl.col(rstd_6h) / (pl.col(rmean_6h).clip(1.0, None))).alias(f'{target}_cv_6h'),
            ])

        if rstd_24h in df.columns and rmean_24h in df.columns:
            df = df.with_columns([
                (pl.col(rstd_24h) / (pl.col(rmean_24h).clip(1.0, None))).alias(f'{target}_cv_24h'),
            ])

        # Volatility ratio (short vs long term)
        if rstd_6h in df.columns and rstd_24h in df.columns:
            # FIXES 20-21 from previous round: Remove spaces in pl. col() and .alias()
            df = df.with_columns([
                (pl.col(rstd_6h) / (pl.col(rstd_24h).clip(0.1, None))).alias(f'{target}_vol_ratio'),
            ])

    # =========================================================================
    # 4. TIME-BASED INTERACTIONS
    # =========================================================================
    print("  [4/7] Creating time-based interaction features...")

    # Hour-specific features
    df = df.with_columns([
        (pl.col('hour_sin') * pl.col('is_rush_hour')).alias('hour_rush_interaction'),
        (pl.col('cos_sza') * pl.col('is_weekend')).alias('sza_weekend_interaction'),
    ])

    if 'is_daytime' in df.columns:
        df = df.with_columns([
            (pl.col('hour_cos') * pl.col('is_daytime')).alias('hour_daytime_interaction'),
        ])

    # Hour of week (0-167)
    df = df.with_columns([
        # FIX 22 from previous round: Remove space in pl. col('dayofweek')
        (pl.col('dayofweek') * 24 + pl.col('hour')).alias('hour_of_week'),
    ])

    df = df.with_columns([
        (pl.col('hour_of_week') * 2 * np.pi / 168).sin().alias('hour_of_week_sin'),
        # FIX 23 from previous round: Remove spaces in pl. col('hour_of_week') * 2 * np. pi / 168). cos()
        (pl.col('hour_of_week') * 2 * np.pi / 168).cos().alias('hour_of_week_cos'),
    ])

    # =========================================================================
    # 5. METEOROLOGICAL INTERACTIONS WITH LAGS
    # =========================================================================
    print("  [5/7] Creating met-lag interaction features...")

    for target in config['targets']:
        lag_1h = f'{target}_lag_1h_iter_{last_iter}'

        if lag_1h in df.columns:
            if 'wind_speed' in df.columns:
                df = df.with_columns([
                    (pl.col(lag_1h) * pl.col('wind_speed')).alias(f'{target}_lag1h_wind'),
                ])

            if 'ventilation' in df.columns:
                # FIX 24 from previous round: Remove space in df. columns
                df = df.with_columns([
                    (pl.col(lag_1h) * pl.col('ventilation')).alias(f'{target}_lag1h_vent'),
                ])

            if 'T_forecast' in df.columns:
                df = df.with_columns([
                    (pl.col(lag_1h) * pl.col('T_forecast')).alias(f'{target}_lag1h_temp'),
                ])

            if 'is_stagnant' in df.columns:
                # FIX 25 from previous round: Remove space in df. columns
                df = df.with_columns([
                    (pl.col(lag_1h) * pl.col('is_stagnant')).alias(f'{target}_lag1h_stagnant'),
                ])

    # =========================================================================
    # 6. DIURNAL POSITION FEATURES
    # =========================================================================
    print("  [6/7] Creating diurnal position features...")

    # Hours since sunrise/sunset proxy
    df = df.with_columns([
        # FIXES 26-27 from previous round: Remove spaces in pl. col('cos_sza') and pl. col('hour')
        pl.when(pl.col('cos_sza') > 0.1)
        .then(pl.col('hour') - 6)
        .otherwise(0)
        .alias('hours_since_sunrise'),
    ])

    df = df.with_columns([
        # FIXES 28-29 from previous round: Remove spaces in pl. col('cos_sza') and pl.col('hour')
        pl.when(pl.col('cos_sza') <= 0.1)
        .then(pl.col('hour') - 18)
        .otherwise(0)
        .alias('hours_since_sunset'),
    ])

    # Peak hour indicators
    df = df.with_columns([
        # FIXES 30-35 from previous round: Remove spaces in pl. col('hour') and pl. Int8 and .alias()
        ((pl.col('hour') >= 12) & (pl.col('hour') <= 16)).cast(pl.Int8).alias('is_o3_peak_hours'),
        ((pl.col('hour') >= 7) & (pl.col('hour') <= 9)).cast(pl.Int8).alias('is_no2_morning_peak'),
        ((pl.col('hour') >= 18) & (pl.col('hour') <= 21)).cast(pl.Int8).alias('is_no2_evening_peak'),
    ])

    # =========================================================================
    # 7. SITE-TEMPORAL INTERACTIONS
    # =========================================================================
    print("  [7/7] Creating site-temporal features...")

    # Site-hour interaction (captures site-specific diurnal patterns)
    for site_id in range(1, config['n_sites'] + 1):
        site_col = f'site_{site_id}'
        if site_col in df.columns:
            df = df.with_columns([
                (pl.col(site_col) * pl.col('hour_sin')).alias(f'site{site_id}_hour_sin'),
                (pl.col(site_col) * pl.col('is_rush_hour')).alias(f'site{site_id}_rush'),
            ])

    new_cols = len(df.columns) - initial_cols
    print(f"\n  Created {new_cols} new features")
    print(f"  Total columns: {len(df.columns)}")

    return df

# Determine last iteration index
# Assuming 'additional_iterations' is defined globally or earlier (e.g., additional_iterations = 3)
last_iter = 2 + additional_iterations - 1  # Should be 4 (0-indexed)

# Create advanced features
df = create_advanced_features(df, CONFIG, last_iter)

# Get final feature list for each target
def get_final_feature_list(df, target, config, last_iter):
    """Get comprehensive final feature list."""
    exclude = [
        'year', 'month', 'day', 'hour', 'date', 'datetime', 'doy',
        'O3_target', 'NO2_target', 'site_id',
        'NO2_satellite', 'HCHO_satellite', 'ratio_satellite',
        'hour_of_week',  # Keep encoded version only
    ]

    target_prefix = target.replace('_target', '')
    other_target = 'NO2' if 'O3' in target else 'O3'

    features = []

    for c in df.columns:
        if c in exclude:
            continue
        if '_oof_iter' in c:
            continue

        # Skip lag features from other target
        if f'{other_target}_target_lag_' in c:
            continue
        if f'{other_target}_target_rmean_' in c:
            continue
        if f'{other_target}_target_rstd_' in c:
            continue
        if f'{other_target}_target_rmin_' in c:
            continue
        if f'{other_target}_target_rmax_' in c:
            continue
        if f'{other_target}_target_range_' in c:
            continue
        if f'{other_target}_target_diff_' in c:
            continue
        if f'{other_target}_target_rate_' in c:
            continue
        if f'{other_target}_target_cv_' in c:
            continue
        if f'{other_target}_target_vol_' in c:
            continue
        if f'{other_target}_target_lag1h_' in c:
            continue

        # For lag features, only keep from last iteration for this target
        if '_iter_' in c and target_prefix in c:
            iter_num = int(c.split('_iter_')[-1])
            if iter_num == last_iter:
                features.append(c)
        elif '_iter_' not in c:
            features.append(c)

    return list(dict.fromkeys(features))

# Prepare final features
final_features = {}
for target in CONFIG['targets']:
    final_features[target] = get_final_feature_list(df, target, CONFIG, last_iter)
    print(f"\n{target}: {len(final_features[target])} final features")

    # Show feature categories
    lag_feats = [f for f in final_features[target] if '_lag_' in f or '_rmean_' in f or '_rstd_' in f]
    # FIX 36 from previous round: Remove space in f. lower()
    ctm_feats = [f for f in final_features[target] if 'forecast' in f.lower() or 'ctm' in f.lower() or 'corr' in f.lower()]
    sat_feats = [f for f in final_features[target] if 'sat' in f.lower()]

    print(f"  - Lag/Rolling: {len(lag_feats)}")
    print(f"  - CTM: {len(ctm_feats)}")
    print(f"  - Satellite: {len(sat_feats)}")
    print(f"  - Other: {len(final_features[target]) - len(lag_feats) - len(ctm_feats) - len(sat_feats)}")

# FIX 37 from previous round: Remove space in df. shape
print(f"\nDataFrame shape: {df.shape[0]:,} x {df.shape[1]}")


STAGE 8: ADVANCED FEATURE ENGINEERING

  [1/7] Creating cross-target features...
  [2/7] Creating lag difference features...
  [3/7] Creating volatility ratio features...
  [4/7] Creating time-based interaction features...
  [5/7] Creating met-lag interaction features...
  [6/7] Creating diurnal position features...
  [7/7] Creating site-temporal features...

  Created 44 new features
  Total columns: 395

O3_target: 172 final features
  - Lag/Rolling: 34
  - CTM: 19
  - Satellite: 14
  - Other: 105

NO2_target: 172 final features
  - Lag/Rolling: 34
  - CTM: 19
  - Satellite: 14
  - Other: 105

DataFrame shape: 171,679 x 395


In [27]:
# Assuming all necessary imports (Polars, NumPy, KFold, CatBoost, tqdm, Optuna, train_test_split) are present.
from sklearn.model_selection import train_test_split

# --- Configuration Setup ---
if 'random_state' not in CONFIG:
    CONFIG['random_state'] = 42
if 'n_folds' not in CONFIG:
    CONFIG['n_folds'] = 5

N_TRIALS = 20
RANDOM_STATE = CONFIG.get('random_state', 42)
N_FOLDS = CONFIG.get('n_folds', 5)

# Cell 3: Hyperparameter Tuning with Optuna (80/20 Split Version)
print("\n" + "=" * 80)
print("STAGE 9: HYPERPARAMETER TUNING WITH OPTUNA (80/20 SPLIT)")
print("=" * 80)

import optuna
from optuna.samplers import TPESampler
# We use WARNING to keep the output clean, letting the TQDM bar show overall progress
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- Data Preparation ---
def prepare_arrays_v3(df, features, target):
    """Prepare X, y arrays for training."""
    df_valid = df.filter(pl.col(target).is_not_null())
    available = [f for f in features if f in df_valid.columns]
    X = df_valid.select(available).to_numpy().astype(np.float32)
    y = df_valid[target].to_numpy().astype(np.float32)
    
    # Median imputation
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np.isnan(col_medians), 0, col_medians)
    for j in range(X.shape[1]):
        X[np.isnan(X[:, j]), j] = col_medians[j]
    
    return X, y, available, col_medians

# --- Objective Factory (Modified for 80/20 Split) ---
def objective_factory(X, y, random_state):
    """
    Create objective function for Optuna using a single 80/20 split.
    This is 5x faster than 5-fold CV for tuning.
    """
    
    # 1. Perform the split ONCE here, outside the trial loop
    #    This ensures every trial uses the exact same validation set for fair comparison.
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.20, random_state=random_state, shuffle=True
    )
    
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 800,1000),
            'depth': trial.suggest_int('depth', 12, 14),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'bootstrap_type': 'Bernoulli',
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'loss_function': 'RMSE',
            'random_seed': random_state,
            'verbose': False,
            'early_stopping_rounds': 100,
            'task_type': 'GPU',
            'devices': '0:1', 
        }
        
        # 2. Train a single model on the split
        model = cb.CatBoostRegressor(**params)
        
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            verbose=False
        )
        
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        
        return rmse
    
    return objective

# --- Tuning Execution ---

tuning_results = {}

for target in tqdm(CONFIG['targets'], desc="Tuning Targets"):
    print(f"\n{'='*60}")
    print(f"TUNING {target}")
    print(f"{'='*60}")
    
    # Prepare data
    features = final_features[target]
    X, y, used_features, col_medians = prepare_arrays_v3(df, features, target)
    
    print(f"  Features: {len(used_features)}, Samples: {len(y):,}")
    
    # Create study
    sampler = TPESampler(seed=RANDOM_STATE)
    study = optuna.create_study(direction='minimize', sampler=sampler)
    
    # Create objective (Using 80/20 split factory)
    objective = objective_factory(X, y, RANDOM_STATE)
    
    # Optimize with progress tracking
    print(f"  Running {N_TRIALS} trials (80/20 Holdout)...")
    
    with tqdm(total=N_TRIALS, desc=f"  {target} Trials", leave=True) as pbar_instance:
        def minimal_callback(study, trial):
            if trial.state.is_finished():
                pbar_instance.update(1)
                pbar_instance.set_postfix({'best_rmse': f"{study.best_value:.4f}"})

        study.optimize(objective, n_trials=N_TRIALS, callbacks=[minimal_callback], show_progress_bar=False)
    
    best_params = study.best_params
    best_rmse = study.best_value
    
    print(f"\n  Best RMSE (Val): {best_rmse:.4f} (Trial {study.best_trial.number})")
    print(f"  Best Parameters:")
    for k, v in best_params.items():
        if isinstance(v, float):
            print(f"    {k}: {v:.6f}")
        else:
            print(f"    {k}: {v}")
    
    # --- FINAL TRAINING (Still using 5-Fold CV) ---
    # We use 5-fold here to generate the robust OOF predictions needed for later stages.
    print(f"\n  Training final model with best params (5-Fold CV)...")
    
    final_params = {
        **best_params, 
        'loss_function': 'RMSE',
        'verbose': False,
        'early_stopping_rounds': 100,
        'task_type': 'GPU', 
        'devices': '0:1',   
        'bootstrap_type': 'Bernoulli', 
    }
    
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    oof_tuned = np.zeros(len(y))
    models_tuned = []
    fold_metrics_tuned = []
    
    # Verbose TQDM for final training folds
    for fold_idx, (train_idx, val_idx) in enumerate(tqdm(kf.split(X), total=N_FOLDS,
                                                         desc=f"  Final Training", leave=False)):
        final_params['random_seed'] = RANDOM_STATE + fold_idx
        model = cb.CatBoostRegressor(**final_params)
        
        model.fit(
            X[train_idx], y[train_idx],
            eval_set=(X[val_idx], y[val_idx]),
            verbose=False
        )
        
        preds = model.predict(X[val_idx])
        oof_tuned[val_idx] = preds
        models_tuned.append(model)
        
        fold_rmse = np.sqrt(mean_squared_error(y[val_idx], preds))
        fold_metrics_tuned.append(fold_rmse)
        
    final_rmse = np.sqrt(mean_squared_error(y, oof_tuned))
    final_mae = mean_absolute_error(y, oof_tuned)
    final_r2 = r2_score(y, oof_tuned)
    
    print(f"\n  FINAL TUNED RESULTS (5-Fold OOF):")
    print(f"    RMSE: {final_rmse:.4f} (+/- {np.std(fold_metrics_tuned):.4f})")
    print(f"    MAE:  {final_mae:.4f}")
    print(f"    R²:   {final_r2:.4f}")
    
    # Compare with baseline
    baseline_rmse = all_iteration_results[target][-1]['rmse']
    improvement = baseline_rmse - final_rmse
    print(f"    Improvement over baseline: {improvement:+.4f}")
    
    tuning_results[target] = {
        'best_params': best_params,
        'final_params': final_params,
        'best_rmse': best_rmse, # Note: This is from the 80/20 split
        'final_rmse': final_rmse, # This is from the full 5-fold OOF
        'final_mae': final_mae,
        'final_r2': final_r2,
        'oof': oof_tuned,
        'models': models_tuned,
        'features': used_features,
        'col_medians': col_medians,
        'y': y,
        'study': study,
    }

# Summary
print("\n" + "=" * 80)
print("HYPERPARAMETER TUNING SUMMARY")
print("=" * 80)

print(f"\n{'Target':<15} {'Baseline':>12} {'Tuned (OOF)':>12} {'Improvement':>12}")
print("-" * 55)

for target in CONFIG['targets']:
    baseline = all_iteration_results[target][-1]['rmse']
    tuned = tuning_results[target]['final_rmse']
    improvement = baseline - tuned
    print(f"{target:<15} {baseline:>12.4f} {tuned:>12.4f} {improvement:>+12.4f}")


STAGE 9: HYPERPARAMETER TUNING WITH OPTUNA (80/20 SPLIT)


Tuning Targets:   0%|          | 0/2 [00:00<?, ?it/s]


TUNING O3_target
  Features: 172, Samples: 171,679
  Running 20 trials (80/20 Holdout)...


  O3_target Trials:   0%|          | 0/20 [00:00<?, ?it/s]


  Best RMSE (Val): 9.1869 (Trial 10)
  Best Parameters:
    iterations: 989
    depth: 13
    learning_rate: 0.093226
    l2_leaf_reg: 9.500492
    min_child_samples: 50
    subsample: 0.915137

  Training final model with best params (5-Fold CV)...


...


  FINAL TUNED RESULTS (5-Fold OOF):
    RMSE: 9.3247 (+/- 0.0740)
    MAE:  5.6766
    R²:   0.9242
    Improvement over baseline: +0.3040

TUNING NO2_target
  Features: 172, Samples: 171,679
  Running 20 trials (80/20 Holdout)...


  NO2_target Trials:   0%|          | 0/20 [00:00<?, ?it/s]


  Best RMSE (Val): 8.3929 (Trial 19)
  Best Parameters:
    iterations: 977
    depth: 13
    learning_rate: 0.074340
    l2_leaf_reg: 1.083437
    min_child_samples: 35
    subsample: 0.714283

  Training final model with best params (5-Fold CV)...


  Final Training:   0%|          | 0/5 [00:00<?, ?it/s]


  FINAL TUNED RESULTS (5-Fold OOF):
    RMSE: 8.4042 (+/- 0.1340)
    MAE:  5.4544
    R²:   0.9174
    Improvement over baseline: +0.2717

HYPERPARAMETER TUNING SUMMARY

Target              Baseline  Tuned (OOF)  Improvement
-------------------------------------------------------
O3_target             9.6287       9.3247      +0.3040
NO2_target            8.6759       8.4042      +0.2717
